# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic


The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [8]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self,
                 min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None):

        self.tree = dict()
        # Дерево реализовано словарём, где ключём
        # элемнта является уникальный идентефикатор узла дерева
        # Значение является кортедж [тип узла, номер аргумета,
        # порог для аргумента]
        # тип узла NON_LEAF_TYPE или LEAF_TYPE

        self.min_samples_split = min_samples_split
        # Минимальное количество элементов выборки в узел, чтобы
        # его можно было делить
        # Иначе вершина

        self.max_depth = max_depth
        # Максимальная глубина дерева

        self.sufficient_share = sufficient_share
        # Минимальная необходимая доля класса в узле, чтобы определить его как
        # Лист этого класса

        self.num_class = -1
        # Кол-во классов

        self.feature_importances_ = None
        # Выбор меры прироста информации

        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
        # Кол-во критериев для анализа

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        return (((1 - (((l_c / l_s) ** 2).sum(axis=1))).
                reshape(-1, 1) * l_s / (l_s + r_s) +
                (1 - (((r_c / r_s) ** 2).sum(axis=1))).
                reshape(-1, 1) * r_s / (l_s + r_s)))

    def __entropy(self, l_c, l_s, r_c, r_s):

        l_c = np.nan_to_num(l_c)
        r_c = np.nan_to_num(r_c)

        l_c /= l_s
        r_c /= r_s

        return (-(l_c * l_c.log2()).sum(axis=1).
                reshape(-1, 1) * l_s / (l_s + r_s) -
                (r_c * r_c.log2()).sum(axis=1).
                reshape(-1, 1) * r_s / (l_s + r_s))

    def __misclass(self, l_c, l_s, r_c, r_s):

        l_c = np.nan_to_num(l_c)
        r_c = np.nan_to_num(r_c)

        l_c /= l_s
        r_c /= r_s

        return (1 - l_c.max(axis=1)).reshape(-1, 1) * l_s / (l_s + r_s) + \
            (1 - r_c.max(axis=1)).reshape(-1, 1) * r_s / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)

        return features_ids[:int(math.sqrt(feature_ids.size))]
        # Выбираем sqrt(N) случайных критериев для анализа

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)

        return features_ids[:int(math.log(feature_ids.size, 2))]
        # Выбираем log2(N) случайных критериев для анализа

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
        # Выбираем все критерии для анализа

    def __sort_samples(self, x, y):
        '''
        Сортирует х и у в порядке возрастания х
        '''

        sorted_idx = x.argsort()

        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        '''
        Делит выборку на две части: левую, значение ключевого атрибута
        которой больше порога и правую состоящую из остальных
        '''

        left_mask = x[:, feature_id] >= threshold
        right_mask = ~left_mask

        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        '''
        Поиск порога критерия для разделения выборки на две части
        '''

        sorted_x, sorted_y = self.__sort_samples(x, y)
        # Сортировка выбранного атрибута и целевого
        # значнения по возрастанию значения атрибута

        class_number = np.unique(y).shape[0]
        # Кол-во классов в выборке

        samples_clip = int(self.min_samples_split / 2)

        splitted_sorted_y = sorted_y[samples_clip:-samples_clip]
        # Выделение крайних значений в сортированой выборке

        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (samples_clip + 1)

        # Выделение позиций значений целовой
        # функции, в которых произошло изменение класса
        # Позиция правого числа в необрезанной сориторованной выборке
        # Отсечённые части не учитываются

        if len(r_border_ids) == 0:
            return np.inf, None

        # Если выборка такая, что в обрезанной части
        # Только одинаковые элементы, то вернуть значение неопределенности
        # В разделённой выборке, следовательно критерий не будет выбран

        eq_el_count = r_border_ids - \
            np.append([samples_clip], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1

        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] += np.bincount(sorted_y[:samples_clip],
                                           minlength=class_number)
        # Создаём таблицу кол-во столбцов - кол-во классов
        # Кол-во строк - кол-во последовательных "кусков"
        # в сотрировнной и обрезаной выборке цел. ф-ии
        # Кусок i состоит из одинаковых значений класса
        # В [i, j] кол-во элеметов класса j в куске i
        # Учитываются все первые эл-ты, в том числе и до границы обрезания
        # Последние (за границей) не учитываются

        l_class_count = np.cumsum(class_increments, axis=0, dtype=np.float32)
        r_class_count = (np.bincount(sorted_y) -
                         l_class_count).astype(np.float32)
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # В [i,j] l_class_count кол-во элементов j
        # го класса в отсортированной выборке
        # до i-го куска включая, в r_class_count - после
        # l_size - координата куска слева (коорд. правого
        # числа в отсортировннаой выборке),
        # r_size - координата куска справа (коорд. лев.
        # числа в отсортировннаой выборке)

        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Считаем меру неопределённости для каждого
        # разбиения и выбираем минимум

        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] +
                         sorted_x[left_el_id]) / 2.0
        # Возвращаем значние неопределённости и порог в сортированном массиве

    def __fit_node(self, X, y, node_id, depth, pred_f=-1):
        ration = np.bincount(y, minlength=3).astype(np.float32)
        ration = ration / ration.sum()
        # Считаем соотношение частей классов в текущей выборке

        if not (self.max_depth is None) and (depth >= self.max_depth) or \
                X.shape[0] <= self.min_samples_split or \
                ration.max() >= self.sufficient_share:

            res = np.argmax(np.bincount(y))
            self.tree.update(
                {node_id: [self.__class__.LEAF_TYPE, res, ration]})
            return

        # Условие узла как листа: глубина больше макс. допустимой глубины или
        # кол-во элементов выборки в узле равно или менее минимального чила
        # Если максимальная доля какого
        # -либо класса больше установленного порога
        # Объявить узел листом

        y_tmp = np.array(y)
        for i in range(np.unique(y).size):
            y[y == np.unique(y)[i]] = i
        # Изменяем значения целевой переменной
        # на числа от 0 до кол-ва уникальных значений

        features_ids = self.get_feature_ids(X.shape[1])
        # Выбор характеристик для анализа

        imp_arr = np.zeros(features_ids.size, dtype=np.float32)
        threshold_arr = np.zeros(features_ids.size, dtype=np.float32)
        # Массив мин. неопределенности и массив порогов для хар-ки

        for i in range(features_ids.size):
            imp_arr[i], threshold_arr[i] = self.__find_threshold(
                X.T[features_ids[i]], y)
        # Поиск мин. неопр. и соответсвующего порога для каждого критерия

        y = y_tmp
        # Восстанавливаем значения целевой переменной

        if np.isinf(imp_arr).all():
            res = np.argmax(np.bincount(y))
            self.tree.update(
                {node_id: [self.__class__.LEAF_TYPE, res, ration]})
            return None
        # Если все значения в разбиение бб
        # возвращаем значение большей части класса

        min_imp_feature = np.argmin(imp_arr)
        # Выбор свойства с наименьшим значение неопределнности

        if np.argwhere(imp_arr == imp_arr.min())[:, 0].size != 1:
            k = np.argwhere(imp_arr == imp_arr.min())[:, 0]
            np.random.shuffle(k)
            min_imp_feature = k[0]
        # Если таких значений несколько, то выбираем случайное

        threshold = threshold_arr[min_imp_feature]
        # Выделение соответсвующего порога

        feature_pos = features_ids[min_imp_feature]
        self.feature_importances_[feature_pos] += 1
        x_l, x_r, y_l, y_r = self.__div_samples(X, y, feature_pos, threshold)
        # Разделение выборки на две части для левого и правого листа

        if x_r.shape[0] == 0 or x_l.shape[0] == 0:
            res = np.argmax(np.bincount(y))
            self.tree.update(
                {node_id: [self.__class__.LEAF_TYPE, res, ration]})
            return None
            # Если одна часть пустая, то лист

        self.tree.update(
            {node_id: [self.__class__.NON_LEAF_TYPE, feature_pos, threshold]})
        # Добавление текущего листа в дерево

        self.__fit_node(x_l, y_l, node_id * 2 + 1, depth + 1)
        self.__fit_node(x_r, y_r, node_id * 2 + 2, depth + 1)
        # Создаём две подвершины

        return None

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [9]:
#%%pycodestyle

my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [10]:
#%%pycodestyle

wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data,
                                                    wine.target,
                                                    test_size=0.1,
                                                    stratify=wine.target)

## Проверка скорости работы на wine

In [11]:
%time clf.fit(X_train, y_train)

CPU times: user 2.25 ms, sys: 277 µs, total: 2.53 ms
Wall time: 8.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
%time my_clf.fit(X_train, y_train)

CPU times: user 29.9 ms, sys: 3.74 ms, total: 33.6 ms
Wall time: 41.1 ms


## Проверка качества работы на wine

In [13]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9407407407407408

In [14]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

## Подготовка данных Speed Dating Data 

In [25]:
#%%pycodestyle

df = pd.read_csv('Speed Dating Data.csv',
                 encoding='cp1251')

In [26]:
#%%pycodestyle

df = df.iloc[:, :97]
# Оставляем первые 97 элементов

df = df.drop(['condtn', 'round', 'position', 'positin1', 'order',
              'partner', 'id', 'idg', 'age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb',
              'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o',
              'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o',
              'field', 'from', 'zipcode', 'career', 'sports', 'tvsports',
              'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts',
              'music', 'shopping', 'yoga', 'expnum', 'undergra', 'field'
              ], axis=1)
# Удаляем ненужные столбцы параметров

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
# Заполняем NaN нулями

tmp = pd.get_dummies(df.loc[:, 'field_cd'], prefix='field_cd', prefix_sep='=')
df = pd.concat([df, tmp], axis=1)
# Кодируем категориальные признаки и конкатенируем таблицы

df = df.dropna(subset=['imprelig', 'imprace', 'age', 'date'])
# Удаляем семплы, где пропущены значения в каком-либо из столбцов выше

df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
# Заполняем NaN малыми числами

df.loc[:, 'tuition'] = df.loc[:,
                              'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
# Заполняем NaN малыми числами

df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
# Заполняем NaN малыми числами

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
# Заполняем NaN нулями

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                        'fun1_1', 'amb1_1',
                                        'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
           'fun1_1', 'amb1_1',
           'shar1_1']] = \
           (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                       'fun1_1', 'amb1_1',
                       'shar1_1']].T / df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                        'fun2_1', 'amb2_1',
                                        'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
           'fun2_1', 'amb2_1', 'shar2_1']] = \
           (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                       'fun2_1', 'amb2_1', 'shar2_1']].T /
            df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)
# Удаляем временную колонку

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)
# Признаки attr4 и attr5 выбросим

df = df.drop(['wave'], axis=1)
# Удаляем временную колонку

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
# Теперь создадим таблицу с мужчинами
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr',
                                          'samerace'], axis=1).dropna()
# Таблицу с женщинами

df_female.columns = df_female.columns + '_f'
# Создадим постфикс для названий колонок

df_female = df_female.drop(['pid_f'], axis=1)
# Удаляем ненужную колонку

df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
# Связываем таблицы

df_pair = df_pair.drop(['iid', 'pid'], axis=1)
# Удаляем ненужную колонку

In [27]:
#%%pycodestyle

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)

## Проверка скорости работы на Speed Dating Data 

In [28]:
%time clf.fit(X_train, y_train)

CPU times: user 191 ms, sys: 120 µs, total: 191 ms
Wall time: 205 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [29]:
%time my_clf.fit(X_train, y_train)

CPU times: user 3.46 s, sys: 3.15 ms, total: 3.46 s
Wall time: 3.59 s


## Проверка качества работы на Speed Dating Data

In [30]:
#%%pycodestyle

clf = DecisionTreeClassifier(min_samples_split=2)
clf.fit(X_train, y_train)
f1_score(y_pred=clf.predict(X_test),
         y_true=y_test.reshape(-1, 1),
         average='macro')

0.5657593180812995

In [31]:
#%%pycodestyle

my_clf = MyDecisionTreeClassifier(min_samples_split=2)
my_clf.fit(X_train, y_train)
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.47548704774138295

## Задание 3

In [32]:
#%%pycodestyle

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)

my_clf = DecisionTreeClassifier(min_samples_split=2)
my_clf.fit(X_train, y_train)

clf = MyDecisionTreeClassifier(min_samples_split=2)
clf.fit(X_train, y_train)

In [33]:
#%%pycodestyle

print(np.argsort(my_clf.feature_importances_)[:-10:-1])
print(np.argsort(clf.feature_importances_)[:-10:-1])

[ 0 72 74 68 70 67 58 17 66]
[ 0 55 20 75 85 56 72 78 11]


## Задание 4

## Задание 5

In [36]:
#%%pycodestyle

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)

my_clf = DecisionTreeClassifier(min_samples_split=2)
my_clf.fit(X_train, y_train)

clf = MyDecisionTreeClassifier(min_samples_split=2)
clf.fit(X_train, y_train)

In [37]:
#%%pycodestyle

parameters = {'n_estimators': np.arange(4, 20),
              'min_samples_split': np.arange(2, 10)}

clf = GridSearchCV(RandomForestClassifier(), parameters)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8225